In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

import pandas_profiling
from tqdm import tqdm_notebook
import numpy as np
%matplotlib inline

/Users/arty/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
import h2o
from h2o.automl import H2OAutoML, get_leaderboard


In [3]:
h2o.init(ip="localhost", port=8889, max_mem_size_GB = 2)

Checking whether there is an H2O instance running at http://localhost:8889 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "13.0.1" 2019-10-15; OpenJDK Runtime Environment (build 13.0.1+9); OpenJDK 64-Bit Server VM (build 13.0.1+9, mixed mode, sharing)
  Starting server from /Users/arty/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/h5/fw31dh0x2hv6pxhyb_zbmw1r0000gn/T/tmp18qtpfzc
  JVM stdout: /var/folders/h5/fw31dh0x2hv6pxhyb_zbmw1r0000gn/T/tmp18qtpfzc/h2o_arty_started_from_python.out
  JVM stderr: /var/folders/h5/fw31dh0x2hv6pxhyb_zbmw1r0000gn/T/tmp18qtpfzc/h2o_arty_started_from_python.err
  Server is running at http://127.0.0.1:8889
Connecting to H2O server at http://127.0.0.1:8889 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_arty_zmy3wl
H2O cluster total nodes:,1
H2O cluster free memory:,2 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [4]:
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("new_train.csv")
test = h2o.import_file("new_test.csv")

# Identify predictors and response
x = train.columns
y = "goal1"
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [6]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%| 100%| 100%| 100%| 100%| 100%| 100%| 100%


In [8]:
lb = aml.leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
XGBoost_3_AutoML_20200104_180047,0.740579,0.0963996,0.102059,0.425709,0.144266,0.0208127,925548,0.012694
XGBoost_grid__1_AutoML_20200104_180047_model_4,0.740043,0.0964911,0.101068,0.412753,0.144301,0.0208228,1.13313e+06,0.010568
XGBoost_grid__1_AutoML_20200104_180047_model_3,0.73922,0.0965241,0.100414,0.417199,0.144345,0.0208356,2.04387e+06,0.011129
XGBoost_grid__1_AutoML_20200104_180047_model_1,0.738541,0.0966608,0.0986974,0.431066,0.144405,0.0208529,835801,0.010682
XGBoost_1_AutoML_20200104_180047,0.733868,0.0968108,0.0986003,0.431517,0.144409,0.020854,9.10366e+06,0.014369
XGBoost_2_AutoML_20200104_180047,0.729548,0.0971681,0.0966287,0.411861,0.144515,0.0208847,1.69381e+06,0.016481
StackedEnsemble_AllModels_AutoML_20200104_180047,0.72836,0.0984306,0.10379,0.428825,0.144824,0.0209739,83074,0.53905
GBM_1_AutoML_20200104_180047,0.727562,0.10084,0.0733745,0.412974,0.148473,0.0220441,68716,0.010523
GBM_3_AutoML_20200104_180047,0.726612,0.0984726,0.0828823,0.42372,0.145744,0.0212412,116813,0.013034
GBM_2_AutoML_20200104_180047,0.726523,0.0984722,0.0827754,0.428245,0.145722,0.0212348,111708,0.010841


In [11]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_3_AutoML_20200104_180047


Model Summary: 


,,number_of_trees
0,,184.0




ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.019686819970991476
RMSE: 0.14030972871113206
LogLoss: 0.088999489906053
Mean Per-Class Error: 0.273629927990936
AUC: 0.8077782548666869
AUCPR: 0.21726176016481785
Gini: 0.6155565097333737

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08563015383222829: 


,,0,1,Error,Rate
0,0,189056.0,2659.0,0.0139,(2659.0/191715.0)
1,1,3320.0,1021.0,0.7648,(3320.0/4341.0)
2,Total,192376.0,3680.0,0.0305,(5979.0/196056.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-06 11:03:17,52 min 11.062 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.977858
1,,2020-01-06 11:03:49,52 min 42.567 sec,5.0,0.398032,0.507256,0.696251,0.108835,10.719543,0.037724
2,,2020-01-06 11:04:13,53 min 6.592 sec,10.0,0.322029,0.386860,0.705959,0.114217,11.077910,0.037642
3,,2020-01-06 11:04:37,53 min 30.885 sec,15.0,0.266152,0.304636,0.713652,0.119229,11.233632,0.037928
4,,2020-01-06 11:05:01,53 min 55.015 sec,20.0,0.225895,0.246729,0.715044,0.124378,11.423375,0.033465
5,,2020-01-06 11:05:25,54 min 19.194 sec,25.0,0.197617,0.205166,0.718142,0.126489,11.653685,0.030588
6,,2020-01-06 11:05:51,54 min 44.791 sec,30.0,0.178303,0.174996,0.719395,0.129185,11.999150,0.033439
7,,2020-01-06 11:06:16,55 min 9.631 sec,35.0,0.165499,0.153028,0.720811,0.131148,11.837933,0.034409
8,,2020-01-06 11:06:40,55 min 34.173 sec,40.0,0.157227,0.137024,0.724159,0.134466,12.206429,0.034240
9,,2020-01-06 11:07:06,55 min 59.618 sec,45.0,0.151923,0.125290,0.726250,0.137115,12.275522,0.033904



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,goal24,7002.184082,1.000000,0.117971
1,field16,4933.279785,0.704534,0.083114
2,vc_field12,4214.362793,0.601864,0.071002
3,next_field0,1859.360229,0.265540,0.031326
4,next_field16,1786.268921,0.255102,0.030094
5,next_first_order_diff,1642.736572,0.234603,0.027676
6,vc_field16,1582.610718,0.226017,0.026663
7,field12,1350.187134,0.192824,0.022748
8,field14,986.311401,0.140858,0.016617
9,field6,774.786560,0.110649,0.013053



See the whole table with table.as_data_frame()


In [23]:
preds = aml.predict(test.drop('goal1'))

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [60]:
preds = aml.leader.predict(test.drop('goal1'))


xgboost prediction progress: |████████████████████████████████████████████| 100%


In [61]:
ans = pd.DataFrame(index=test['orderid'].as_data_frame().squeeze() - 196056)
ans['proba'] = preds['p1'].as_data_frame().squeeze().values
ans.sort_index(inplace=True)
ans.to_csv('h2o.csv')